In [2]:
from terracatalogueclient import Catalogue
import datetime as dt
import glob
import pandas as pd
import geopandas as gpd
import sys, os

from util import suppress_stdout, create_gpd_for_scene

In [6]:
def create_reference_scene_json(start, end, aoi_file: str = None, bursts_file: str = None):
    """Create a GeoJSON that contains a set of Sentinel 1 reference scenes that are needed as common coregister-references.
    
    This function employs some tests to make sure every individual scene is only covered once. 
    However the file coming out of this should be checked one in a while.
    
    :param start: The start time of the search
    :param end: The end time of the search
    :param aoi_file: AOI file to limit the search
    :param bursts_file: existing .geojson file with reference scenes
    """
    
    # input checks
    timediff = start - end
    if not timediff == dt.timedelta(-12):
        print("[CAUTION]: For full coverage, a 12 day timedelta is needed.")
    
    if os.path.isfile(aoi_file):
        aoi = gpd.read_file(aoi_file)
    else:
        print("[ERROR]: no aoi_file given")
        return
    
    ref_inter_bursts_file = bursts_file
    if os.path.isfile(ref_inter_bursts_file):
        ref_inter_bursts = gpd.read_file(ref_inter_bursts_file)
        create_new_file = False
    else:
        print("ref_inter_bursts_file not found")
        create_new_file = True

    print("starting search for scenes...")

    catalogue = Catalogue()

    cat = catalogue.get_products(
        "urn:eop:VITO:CGS_S1_SLC_L1",
        start = start,
        end = end
        # , geometry =  WKT string or shapely geom
    )

    s1a = []

    for p in cat:
        path = p.data[0].href 
        iw_index = path.index("IW")
        vm_path = "/data/MTDA/CGS_S1/CGS_S1_SLC_L1/" + path[iw_index:]

        # make swath geometry and add basic info to df
        ana_split = create_gpd_for_scene(vm_path, make_regular = True)

        # append to list based on satellite
        if ana_split["sensor"].iloc[0] == "S1A":

            # AOI INTERSECTION
            # create boolean of which bursts intersect with aoi and which dont
            intersects = ana_split.intersects(aoi.iloc[0]["geometry"])
            # keep only those bursts that intersect with aoi
            intersecting_bursts = ana_split[intersects]

            if not intersecting_bursts.empty:

                # CHECK IF EXISTS
                rel_o = intersecting_bursts.iloc[0]["rel_orbit"]
                o_dir = intersecting_bursts.iloc[0]["orbit_direction"]

                # if a file already exists
                if not create_new_file:
                    # search in existing table for bursts of the same relative orbit and orbit direction
                    check = ref_inter_bursts.loc[(ref_inter_bursts["rel_orbit"] == rel_o) & (ref_inter_bursts["orbit_direction"] == o_dir)]
                    # if some are found:
                    if not check.empty:
                        # intersect with the new bursts
                        intersec = gpd.overlay(intersecting_bursts, check, how = "intersection") # .to_file("intersection_test_" + ana_split.iloc[0]["id"] + ".geojson", driver = "GeoJSON")
                        intersec["area"] = intersec.to_crs({'init': 'epsg:32631'}).area
                        # calculate the overall intersecting area
                        intersec_area = intersec["area"].sum()
                    else:
                        # if none are found, intersecting area is 0
                        intersec_area = 0

                    # calculate area of new bursts
                    new_scene_area = intersecting_bursts.to_crs({'init': 'epsg:32631'}).area.sum()
                    # calculate ratio between the two areas
                    ratio = intersec_area / new_scene_area

                # otherwise add all, of course
                else:
                    ratio = 0

                if ratio < 0.9:
                    s1a.append(intersecting_bursts)

    if s1a:
        s1a_df = gpd.GeoDataFrame(pd.concat(s1a, ignore_index = True), crs=s1a[0].crs)

        # if a file exists, add to it and rewrite
        if not create_new_file:
            print(len(s1a_df), " bursts added.")
            s1a_df = ref_inter_bursts.append(s1a_df, sort = False)
        else:
            print(len(s1a_df), " bursts found.")

        # create empty dictionairy for the mapping ID - frame number
        relative_frames = {}
        # initiate column
        s1a_df["rel_frame"] = 0
        # init frame number
        fnr = int(1)

        # TODO heard that iterrows() is slow, not sure how I could improve here
        for index, row in s1a_df.iterrows():
            if row["id"] in relative_frames:
                s1a_df.at[index, "rel_frame"] = relative_frames[row["id"]]
            else:
                s1a_df.at[index, "rel_frame"] = fnr
                relative_frames[row["id"]] = fnr
                fnr += 1

        # write bursts
        s1a_df.to_file("reference_bursts.geojson", driver = "GeoJSON")
        # extract scenes and write
        s1a_df.dissolve(["id"], as_index = False).to_file("reference_scenes.geojson", driver = "GeoJSON")

    elif not s1a:
        print("nothing added")
    else:
        print("error")

    # gpd.overlay(aoi, s1a_df, how = "intersection").to_file("test2.geojson", driver = "GeoJSON")

    print("end")
    return



# input time frame in which reference scenes should be defined
# this should be no longer than 12 days! after 12 days, orbits of a single satellite repeat and ambiguities arise
# My use case was to collect the base scenes from 1.10 - 12.10.2021, and to add some scenes over france from oct 2020 later on
start = dt.date(2020, 10, 1)
end = dt.date(2020, 10, 13)
bursts_file = "/home/jonathanbahlmann/Public/coherence-docs/src/reference_bursts.geojson"
aoi_file = "/home/jonathanbahlmann/Public/coherence-docs/aoi/belgium_france.geojson"

create_reference_scene_json(start = start, end = end, aoi_file = aoi_file, bursts_file = bursts_file)

starting search for scenes...


/usr/local/lib/python3.6/site-packages/geopandas/base.py:35: UserWarning: GeoSeries crs mismatch: EPSG:4326 and {'init': 'epsg:4326'}
  right.crs))


scene  1D76 , IW3, has  10  bursts.
scene  0A0F , IW2, has  10  bursts.
scene  B9E7 , IW3, has  10  bursts.
scene  9046 , IW2, has  10  bursts.
scene  83DA , IW1, has  12  bursts.
scene  83DA , IW2, has  12  bursts.
scene  83DA , IW3, has  13  bursts.
scene  E320 , IW1, has  10  bursts.
nothing added
end


In [3]:
import os
from util import list_days

# 2021/07/03 - 2021/09/10
start  = "2021/07/03"
end = "2021/11/10"
path = "/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/"

def list_products_by_time(start, end, path = "/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/"):
    s_month = int(start[5:7])
    e_month = int(end[5:7])
    s_day = int(start[8:])
    e_day = int(end[8:])

    start_folder = path + start[0:7] # year and month, no /
    end_folder = path + end[0:7]

    # if only in one month
    if s_month == e_month:
        start_folder_days = os.listdir(start_folder)[s_day-1:e_day] # list days, from start day : end day
        list_of_products = list_days(list_of_days = start_folder_days, month_path = start_folder)
    # no in between month
    elif s_month + 1 == e_month:
        start_folder_days = os.listdir(start_folder)[s_day-1:] # list days, from start day
        end_folder_days = os.listdir(end_folder)[:e_day]
        list_of_products = list_days(start_folder_days, start_folder) + list_days(end_folder_days, end_folder)
    # with month in between
    elif s_month +1 < e_month:
        start_folder_days = os.listdir(start_folder)[s_day-1:]
        end_folder_days = os.listdir(end_folder)[:e_day]
        list_of_products = list_days(start_folder_days, start_folder) + list_days(end_folder_days, end_folder)

        for i in range(s_month+1,e_month):
            if i < 10:
                month = "0" + str(i)
            elif i >= 10:
                month = str(i)

            month_path = os.path.join(path, start[0:5], month)
            month_days_list = os.listdir(month_path)
            month_prod_list = list_days(month_days_list, month_path)
            list_of_products.extend(month_prod_list)
        
    return list_of_products

# print(list_days(["01", "02", "03"], path + "2021/08"))

In [20]:
import geopandas as gpd
from util import create_gpd_for_scene

products = list_products_by_time(start = "2021/09/06", end = "2021/09/10")
# we get 3 products for that day
prod_file = ""

if os.path.isfile(prod_file):
    prod_gpd = gpd.read_file(prod_file)
    create_new_file = False
else:
    print("ref_bursts_file not found")
    create_new_file = True
    
ref_bursts = gpd.read_file("reference_bursts.geojson")
    
for path in products:
    # make swath geometry and add basic info to df
    scene_bursts = create_gpd_for_scene(path)
    
    if create_new_file: #not create_new_file:
        # check every scene
        pass
    
        # check which bursts are eligible for processing (overlap AOI) OR search directly for ref scene
        ref_scene_id = search_for_reference(scene_bursts, ref_bursts)
        print(ref_scene_id)
        
        # can come back empty, len 1 or len 2
        
        
        # check if all bursts are in the file already
        
        # if yes, go ahead
        
        # if no, process
    
    
    
    else:
        # add all scenes
        pass
    
    # write to file

ref_bursts_file not found
['6471']
['3785']
['ACC4']
['7BC8']
['4712', '9F0A']
['7BC8', '9F0A']
['1F6C']
['7E94']
['1F6C']
[]
['AAC4', '3F3B']
['AAC4']
[]
['FB38']
['14F6']
['314F']
['7AA0']
['7AA0', 'A8C7']
['A8C7']
[]
[]


In [44]:
os.path.join("/data", "hello")

'/data/hello'

In [61]:
l1 = ["01", "02"]
l2 = ["01", "03"]
set(l1) == set(l2)
set(l2)

{'01', '03'}